First reformat Mummer output to a human readable txt file.

In [1]:
!../reformating_mummer_results/ReformatMUMMERoutput.sh ../raw_data/LvSf80.out LvSf80.out.tsv

In [201]:
def reformat_mummer(mummer_out_file):
    
    outlist = []
    temp_list = []
    
    for line in open(mummer_out_file, 'r'):
        line=line.strip()
        if line.startswith(">"):
            temp_list = [line.replace("> ","")]
        else:
            temp_list.append('\t'.join(line.split()))
            if len(temp_list) == 3:
                outlist.append("\t".join(temp_list))
                temp_list = [temp_list[0]]
            
    return outlist
            
            
def output_mummer_tabular(outfile, result_list):
    
    fh = open(outfile,'w')
    for res in result_list:
        fh.write(res+'\n')
        
    

In [147]:
def get_ctg_lengths(fasta):
    
    from Bio import SeqIO
    lengths = {}

    for seq in SeqIO.parse(open(fasta,'r'),'fasta'):
        lengths[seq.id]=len(seq.seq)
        
    return lengths

In [168]:
def evaluate_dist_from_end(dist_from_edge, pos, mode, total_length=0):
    
    import sys
    
    ok = True
    
#    dist_from_edge = int(dist_from_edge)
#    pos=int(p)
    if mode == 'start':
        if not pos >= (dist_from_edge): #if match is not far enough from the start of the sequence
            ok = False
        
    elif mode == 'end':
     
        if not total_length > 0:
            sys.exit('need to know the length of the contig')
            
        if not (total_length-pos) >= dist_from_edge:
            ok = False
        
    if ok:
        return True
    else:
        return False

In [172]:
lis = get_ctg_lengths('../raw_data/Lvar_final.contigs.fa')


In [183]:
dist=10
p=3154
ctg_id='k101_421826'
print lis[ctg_id]
evaluate_dist_from_end(dist_from_edge=dist, pos=p, mode='end', total_length=lis[ctg_id])


3163


False

In [116]:
def reformat_match(ref_id, ref_start_pos, baselist):
    
    #ref_id is the unique id of the reference contig
    #ref_start_pos is the coordinate of the start position on the reference
    #baselist is a list containing [query_id, query_start_pos, match_length, sequence]
    #returns a dictionary of format:
    #{'ref':{'id':'xyz', 'start': '123', 'end': '234', 'or':'+', 'seq':'AGCT'}, 
    #'que':{'id':'xyz', 'start': '123', 'end': '234', 'or':'-', 'seq':'AGCT'}}
    
    temp_dict = {'ref':{}, 'que':{}}
    
    temp_dict['ref']['id'] = ref_id
    temp_dict['ref']['start'] = ref_start_pos
    temp_dict['ref']['end'] = ref_start_pos + int(baselist[2])
    temp_dict['ref']['seq'] = baselist[3].upper()
    temp_dict['ref']['or'] = '+'
    temp_dict['que']['id'] = baselist[0]
    temp_dict['que']['start'] = int(baselist[1])
    temp_dict['que']['end'] = int(baselist[1]) + int(baselist[2])
    temp_dict['que']['seq'] = baselist[3].upper()
    temp_dict['que']['or'] = '+'
                
    if baselist[0].endswith('Reverse'):
        temp_dict['que']['id'] = temp_dict['que']['id'].replace('_Reverse','')
        temp_dict['que']['or'] = '-'

    return temp_dict

Write report for the binned matches to file.

In [117]:
def write_binned_match_details(outfile, master_dict):
    
    order = ['id','start','end','or','seq']
        
    ## Write out coordinates of candidates
    outfh = open(outfile,'w')
    for group in master_dict.keys():
        for uce in master_dict[group]:
            linestart = "%s\t%s\t" %(uce,group)
            if isinstance(master_dict[group][uce], list):
#                print uce,len(binned[group][uce])
                for rep in master_dict[group][uce]:
                    temp = []
                    for target in ['ref','que']:
                        for key in order:
                            temp.append(str(rep[target][key]))
                    line=linestart+"\t".join(temp)
                    outfh.write(line+'\n')
            else:
                temp = []
                for target in ['ref','que']:
                    for key in order:
                        temp.append(str(master_dict[group][uce][target][key]))
                line=linestart+"\t".join(temp)
                outfh.write(line+'\n')
    
    outfh.close()

Write valid UCE coordinates to file.

In [118]:
def write_UCE_coordinates(outfile, master_dict):
    
    order = ['id','start','end','or','seq']
        
    ## Write out coordinates of candidates
    outfh = open(outfile,'w')
    for group in master_dict.keys():
        for uce in master_dict[group]:
            line = "%s\t%s\t" %(uce,group)
            temp = []
            for target in ['ref','que']:
                for key in order:
                    temp.append(str(master_dict[group][uce][target][key]))
            line+="\t".join(temp)
            outfh.write(line+'\n')
    
    outfh.close()


Extract UCE sequences to file with or without extensions.

In [224]:
def extract_UCEs(prefix, candidates, target='ref', extension=0, fasta='', minlength=0):
    """
    Write UCE's to fasta file optionally including and extension of x bp up and downstream of the UCE
    """

    import sys
    from Bio import SeqIO

    if extension and not fasta:
        sys.exit("\nYou want to extract UCE's with and extension - need a fasta file for that\n")
    
    if not extension:
        outfh = open(prefix+'.fasta','w')
        for group in candidates.keys():
            for uce in candidates[group]:
                if not len(candidates[group][uce]['ref']['seq']) >= minlength:
                        continue
                header_suffix = '%s|%s|%s' %(uce, candidates[group][uce][target]['id'], candidates[group][uce][target]['start'])
                outfh.write(">%s|%s\n%s\n" %(group, header_suffix, candidates[group][uce][target]['seq']))
    
        outfh.close()
    
    else:
        #identify ref contigs for which I need to extract sequences
        ctgs = {}

        for group in candidates.keys():
            for uce in candidates[group]:
                
                ctg_id = candidates[group][uce][target]['id']
                if not ctg_id in ctgs:
                    ctgs[ctg_id] = []
                ctgs[ctg_id].append(uce+' '+group)
        
        
        outfh = open(prefix+'.fasta','w')
        for seq in SeqIO.parse(open(fasta,'r'),'fasta'):
            if seq.id in ctgs:
        #        print seq.id
                for uce_with_group in ctgs[seq.id]:
                    (uce,group) = uce_with_group.split(" ")
                    if not len(candidates[group][uce]['ref']['seq']) >= minlength:
                        continue
                        
                    startadd = ''
                    endadd = ''
                    
        #            print uce,group
                    start = candidates[group][uce][target]['start']-extension-1
                    end = candidates[group][uce][target]['end']+extension-1
                    if start < 0:
#                        print group,uce,candidates[group][uce][target]
                        startadd = 'N' * (start*-1)
                        start = 0
                
                    if end > len(seq.seq):
    #                    print uce,candidates[group][uce][target]
    #                    print end,len(seq.seq)
                        endadd = 'N' * (end-len(seq.seq))
                    
                    if candidates[group][uce][target]['or'] == '+':
                        uc_seq = str(seq.seq[start:end])
                    else:
                        uc_seq = str(seq.seq.reverse_complement()[start:end])
        #            print "\t%s,%s,%s" %(uce, candidates['merged'][uce]['que']['start'], candidates['merged'][uce]['que']['end'])
                    
                    header_suffix = '%s|%s|%s' %(uce, candidates[group][uce][target]['id'], candidates[group][uce][target]['start'])
                    outfh.write(">%s|%s\n%s\n" %(group, header_suffix, candidates[group][uce][target]['seq']))
                    outfh.write(">%s|%s\n%s%s%s\n" %(group, header_suffix, startadd, uc_seq, endadd))
    
        outfh.close()




Then process the file.

In [234]:

mummer_result = "../raw_data/LvSf80.out" #format of the input here was defined by us (see separate workflow)
#Tab delimited file
#6 columns
#col 1: query contig id
#col 2: REf contig id
#col 3: ref start pos
#col 4: query start pos
#col 5: length of match
#col 6: nucleotide sequence
#ref = subject in mummer, i.e. the fasta that was specified first in the mummer command
#query = query in mummer, i.e. the fasta that was specified second in the mummer command
#example mummer command: mummer -maxmatch -l 120 -F ref.fasta query.fas
#The input file was created from mummer output with a custom script

#Input files - reference and query genome assembly
quefasta = '../raw_data/Sfran_final.contigs.fa'
reffasta = '../raw_data/Lvar_final.contigs.fa'

max_merge_distance = 40 #maximum distance between 2 unique perfect matches from mummer to be merged to a single bait
unique_distance = 1000 #minimum distance between 2 candidates (if distance is shorter only consider the longer perfect match)
dist_from_edge = 10
min_diff = 0 #negative value means that the gap on the query is smaller than on the reference
max_diff = 10 #maximum gap size difference between ref and query in a case of adjacent candidates 

extension=500
minlength=100

###########################################

mummerlist = reformat_mummer(mummer_result)

mummer_result_prefix = mummer_result.split("/")[-1]
print mummer_result_prefix+'.tsv'
output_mummer_tabular(mummer_result_prefix+'.tsv', mummerlist)


mummer_fh = open(mummer_result_prefix+'.tsv','r')
#mummer_fh = open('../reformating_mummer_results/test.out.tsv','r')

per_ref_ctg = {}
total = 0

candidates = {'singlematch':{}, 'unique':{}, 'merged':{}}
binned = {'position':{}, 'too-close':{}, 'overlap-filter':{}, 'skip': {}, 'irregular-merge':{}, 'merge-dist-diff':{}}


que_prefix = ".".join(quefasta.split("/")[-1].split(".")[:-1])
ref_prefix = ".".join(reffasta.split("/")[-1].split(".")[:-1])
global_prefix = mummer_result_prefix

lengths = get_ctg_lengths(reffasta)

#read in reformatted mummer output
#The loop produces a dictionary with the following structure:
#key = reference contig id
#value = dictionary with key = start position on reference and value is list with [queryid, querystart, matchlength, sequence]

for line in mummer_fh:
    llist = line.strip().split("\t")
    total+=1
#    print llist
    if not llist[1] in per_ref_ctg:
        per_ref_ctg[llist[1]] = {}
    if not int(llist[2]) in per_ref_ctg[llist[1]]:
        per_ref_ctg[llist[1]][int(llist[2])] = []
    per_ref_ctg[llist[1]][int(llist[2])].append([llist[0],llist[3],llist[4],llist[5]])

print "Total number of MUMMER matches: %i" %total
#print per_ref_ctg    
    
#Filter 'multiple query hits with identical start position on reference'
#searches for multiple matches with the same startposition on the same reference contig, i.e. more than one matches with the query start at the 
#same position in the reference

filtercount=0
for ref_ctg_id in sorted(per_ref_ctg):
#    print ref_ctg_id
    for ref_ctg_pos in sorted(per_ref_ctg[ref_ctg_id]):
        replicates = len(per_ref_ctg[ref_ctg_id][ref_ctg_pos])
        if replicates > 1:
#            print "\nReference startpoint '%s' on reference ctg id '%s' is hit by %s queries -> filter" %(ref_ctg_pos,ref_ctg_id,replicates)

            matchid = ref_ctg_id+"|%s" %ref_ctg_pos
            binned['position'][matchid] = []
            for rep in per_ref_ctg[ref_ctg_id][ref_ctg_pos]:

                binned['position'][matchid].append(reformat_match(ref_id=ref_ctg_id, ref_start_pos=ref_ctg_pos, baselist=rep))
                
                filtercount+=1
                
            del per_ref_ctg[ref_ctg_id][ref_ctg_pos]
            if len(per_ref_ctg[ref_ctg_id]) == 0:
                del per_ref_ctg[ref_ctg_id]

                
print "Filter 'multiple query hits with identical start position on reference' removed %i Mummer hits" %filtercount
total-=filtercount
print "Leaves us with %i hits to process" %total


##extract uniques
#print "Number of contigs in dictionary before unique matches were removed: %s" %len(per_ref_ctg)
#identify the matches 
for ref_ctg_id in sorted(per_ref_ctg):
#    print ref_ctg_id,len(per_ref_ctg[ref_ctg_id]),sorted(per_ref_ctg[ref_ctg_id])
    if len(per_ref_ctg[ref_ctg_id]) == 1:
        ref_ctg_pos = per_ref_ctg[ref_ctg_id].keys()[0]
        temp = per_ref_ctg[ref_ctg_id][ref_ctg_pos][0]

        
        #check if distance from ctg start is ok
#        if not ref_ctg_pos >= (dist_from_edge): #if match is not far enough from the start of the sequence
        start = evaluate_dist_from_end(dist_from_edge=dist_from_edge, pos=ref_ctg_pos, mode='start')
        end = evaluate_dist_from_end(dist_from_edge=dist_from_edge, pos=ref_ctg_pos, mode='end', total_length=lengths[ref_ctg_id])
        if not start or not end:
#        if not evaluate_dist_from_end(dist_from_edge=dist_from_edge, pos=ref_ctg_pos, mode='start'):
            temp.append('too-close')
#            print "too close to edge - %s" %ref_ctg_id
            del per_ref_ctg[ref_ctg_id]
            continue
            
        else: #move match to candidate status
            
            matchid = ref_ctg_id+"|%s" %ref_ctg_pos
#            print "sending to candidates -> %s" %matchid

            candidates['singlematch'][matchid] = reformat_match(ref_id=ref_ctg_id, ref_start_pos=ref_ctg_pos, baselist=temp)
    
#            print "unique - %s" %ref_ctg_id
                    
#        print "%s\n" %candidates['singlematch'][matchid]

        #remove match from original dictionary
        del per_ref_ctg[ref_ctg_id]

#    print "more than one - %s" %ref_ctg_id


#print "Number of contigs in dictionary after unique matches were removed: %s" %len(per_ref_ctg)
################################################

to_merge = {}


#Identify matches in close proximity
#At this stage the per_ref_ctg dictionary contains exclusively reference contigs that had more than one match with the query, but none 
#that started at the same position (see filter 'multiple query hits with identical start position on reference' above)



for ref_ctg_id in sorted(per_ref_ctg):
    pos_list = sorted(per_ref_ctg[ref_ctg_id])
#    print pos_list
#    print ref_ctg_id,len(per_ref_ctg[ref_ctg_id]),sorted(per_ref_ctg[ref_ctg_id])
#    print "\n### %s" %ref_ctg_id #,len(per_ref_ctg[ref_ctg_id]),sorted(per_ref_ctg[ref_ctg_id])

    for i in range(len(pos_list)-1):
#        print per_ref_ctg[ref_ctg_id][pos_list[i]][0]
        if len(per_ref_ctg[ref_ctg_id][pos_list[i]][0]) == 4:
            if pos_list[i] >= (dist_from_edge):
                per_ref_ctg[ref_ctg_id][pos_list[i]][0].append('unique') #only if it's the first match on a contig ][pos_list[i]][0][4]
            else:
                per_ref_ctg[ref_ctg_id][pos_list[i]][0].append('too-close')
                
#        print "\n[%i]: compare '%i' vs. %i" %(i, pos_list[i], pos_list[i+1])

        #calculate the distance between the current match [i] and the next match [i+1] on the reference contig = distance
        current_length = int(per_ref_ctg[ref_ctg_id][pos_list[i]][0][2])
        distance = pos_list[i+1] - (pos_list[i]+current_length)
#        print "%s: %s %i %s %s" %(ref_ctg_id,pos_list[i],current_length, pos_list[i+1], distance)

        #if the distance between two adjacent matches is larger than a defined value (unique_distance) the match will be considered as unique
        if distance >= unique_distance:
#            print "Unique distance"
            per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('unique')
#            if per_ref_ctg[ref_ctg_id][pos_list[i]][0][4] == 'unique':
#                print pos_list[i],per_ref_ctg[ref_ctg_id][pos_list[i]][0]
#                continue
        
        elif distance < 0:
#            print "overlapping match -> exclude"
            per_ref_ctg[ref_ctg_id][pos_list[i]][0][4] = 'overlap-filter'
            per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('overlap-filter')
        
        elif distance > 0:
            if distance <= max_merge_distance:
#            print "Matches within merge distance"
                if per_ref_ctg[ref_ctg_id][pos_list[i]][0][4] == 'unique':
                    per_ref_ctg[ref_ctg_id][pos_list[i]][0][4] = 'merge-'+ref_ctg_id+'|'+str(pos_list[i])
                    if not ref_ctg_id+'|'+str(pos_list[i]) in to_merge:
                        to_merge[ref_ctg_id+'|'+str(pos_list[i])] = []
                    to_merge[ref_ctg_id+'|'+str(pos_list[i])].append(pos_list[i])
                    
                    per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('merge-'+ref_ctg_id+'|'+str(pos_list[i]))
                    to_merge[ref_ctg_id+'|'+str(pos_list[i])].append(pos_list[i+1])
                elif per_ref_ctg[ref_ctg_id][pos_list[i]][0][4].startswith('merge'): #merge label found -> append same merge label to next 
                    per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append(per_ref_ctg[ref_ctg_id][pos_list[i]][0][4])
                    to_merge[per_ref_ctg[ref_ctg_id][pos_list[i]][0][4].replace('merge-','')].append(pos_list[i+1])
                elif per_ref_ctg[ref_ctg_id][pos_list[i]][0][4] == 'overlap-filter':
                    per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('overlap-filter')
                elif per_ref_ctg[ref_ctg_id][pos_list[i]][0][4] == 'too-close':
                    per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('too-close')
                elif per_ref_ctg[ref_ctg_id][pos_list[i]][0][4] == 'skip':
#                    print "### SKIP CASE"
#                    print pos_list[i+1],per_ref_ctg[ref_ctg_id][pos_list[i+1]],pos_list[i-1],per_ref_ctg[ref_ctg_id][pos_list[i-1]]
#                    print "distance: %s vs. %s" %(pos_list[i+1] - (pos_list[i-1]+int(per_ref_ctg[ref_ctg_id][pos_list[i-1]][0][2])), unique_distance)
                    if (pos_list[i+1] - (pos_list[i-1]+int(per_ref_ctg[ref_ctg_id][pos_list[i-1]][0][2]))) >= unique_distance:
                        per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('unique')
#                        print "i+1 will be unique"
                    else:
                        per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('skip')
#                        print "i+1 will be skip"
                        
            else:
                if per_ref_ctg[ref_ctg_id][pos_list[i]][0][4] == 'unique' or per_ref_ctg[ref_ctg_id][pos_list[i]][0][4].startswith('merge'):
                    per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('skip')
                elif per_ref_ctg[ref_ctg_id][pos_list[i]][0][4] == 'skip':
#                    print "### SKIP CASE 2"
#                    print pos_list[i+1],per_ref_ctg[ref_ctg_id][pos_list[i+1]],pos_list[i-1],per_ref_ctg[ref_ctg_id][pos_list[i-1]]
#                    print "distance: %s vs. %s" %(pos_list[i+1] - (pos_list[i-1]+int(per_ref_ctg[ref_ctg_id][pos_list[i-1]][0][2])), unique_distance)
                    if (pos_list[i+1] - (pos_list[i-1]+int(per_ref_ctg[ref_ctg_id][pos_list[i-1]][0][2]))) >= unique_distance:
                        per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('unique')
#                        print "i+1 will be unique 2"
                    else:
                        per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('skip')
#                        print "i+1 will be skip 2"
                else:
                    per_ref_ctg[ref_ctg_id][pos_list[i+1]][0].append('unique')
                                    
                
#        print pos_list[i],per_ref_ctg[ref_ctg_id][pos_list[i]][0]

        if per_ref_ctg[ref_ctg_id][pos_list[i]][0][4] == 'unique':
            matchid = ref_ctg_id+"|%s" %pos_list[i]
            temp = per_ref_ctg[ref_ctg_id][pos_list[i]][0]
#            print "sending to candidates -> %s" %matchid
        
            candidates['unique'][matchid] = reformat_match(ref_id=ref_ctg_id, ref_start_pos=pos_list[i], baselist=temp)
            
#            print "%s\n" %candidates['unique'][matchid]
    
    #output the decision for the last match on the contig
#    print "[%i]: %i - %s" %(len(pos_list), pos_list[-1], per_ref_ctg[ref_ctg_id][pos_list[-1]][0]) 
    if per_ref_ctg[ref_ctg_id][pos_list[-1]][0][4] == 'unique':
        matchid = ref_ctg_id+"|%s" %pos_list[-1]
        temp = per_ref_ctg[ref_ctg_id][pos_list[-1]][0]
#        print "sending to candidates -> %s" %matchid

        candidates['unique'][matchid] = reformat_match(ref_id=ref_ctg_id, ref_start_pos=pos_list[-1], baselist=temp)            
#        print "%s\n" %candidates['unique'][matchid]


print "\nMERGING\n"

#print to_merge

for mergers in sorted(to_merge):
    ok = True
#    print "\n"+mergers,to_merge[mergers]
    ctg_id = "|".join(mergers.split("|")[:-1])
#    print ctg_id
    temp = []
    query_ids = []

    for start in to_merge[mergers]:
#        print start,per_ref_ctg[ctg_id][start][0]
        temp.append(per_ref_ctg[ctg_id][start][0])
        query_ids.append(per_ref_ctg[ctg_id][start][0][0])

    if len(list(set(query_ids))) == 1: #this is true if all query matches are on the same contig
#        print "single queryID"
        pass
        
    else:
#        print "non unique query ctg"
        
#        print "\n"+mergers,to_merge[mergers]
        binned['irregular-merge'][mergers] = []
        for start2 in to_merge[mergers]:
#            print ctg_id,start2,per_ref_ctg[ctg_id][start2][0]
            binned['irregular-merge'][mergers].append(reformat_match(ref_id=ctg_id, ref_start_pos=start2, baselist=per_ref_ctg[ctg_id][start2][0]))

        continue
    
    for i in range(len(temp)-1):
#        print temp[i]
        distance_ref = to_merge[mergers][i+1] - (to_merge[mergers][i] + int(temp[i][2]))
        distance_que = int(temp[i+1][1]) - (int(temp[i][1]) + int(temp[i][2]))
#        print "REFERENCE DISTANCE: %i" %distance_ref
#        print "QUERY DISTANCE: %i" %distance_que

#        diff = distance_que-distance_ref
        diff = abs(distance_que-distance_ref)
#        print "DIFF: %i\n" %diff
        
#        if diff >= min_diff and diff <= max_diff:

        if diff <= max_diff:
            pass
        else:
#            print "Diff conflict - diff = %i" %diff
            binned['merge-dist-diff'][mergers] = []
            for start2 in to_merge[mergers]:
                binned['merge-dist-diff'][mergers].append(reformat_match(ref_id=ctg_id, ref_start_pos=start2, baselist=per_ref_ctg[ctg_id][start2][0]))

            ok = False
            break
    
    if ok:
#        print "GO MERGE"
        pass
    else:
        continue
    
    ##Merge
    candidates['merged'][mergers] = {'ref':{}, 'que':{}}
#    print candidates['merged'][mergers]
    
    candidates['merged'][mergers]['ref']['id'] = ctg_id
    candidates['merged'][mergers]['ref']['start'] = to_merge[mergers][0]
    candidates['merged'][mergers]['ref']['end'] = to_merge[mergers][-1] + int(temp[-1][2])
    candidates['merged'][mergers]['ref']['seq'] = ''
    candidates['merged'][mergers]['ref']['or'] = '+'
    candidates['merged'][mergers]['que']['id'] = query_ids[0]
    candidates['merged'][mergers]['que']['start'] = int(temp[0][1])
    candidates['merged'][mergers]['que']['end'] = int(temp[-1][1]) + int(temp[-1][2])
    candidates['merged'][mergers]['que']['seq'] = ''
    candidates['merged'][mergers]['que']['or'] = '+'
    
    if query_ids[0].endswith('Reverse'):
        candidates['merged'][mergers]['que']['or'] = '-'
        candidates['merged'][mergers]['que']['id'] = query_ids[0].replace('_Reverse','')

#    print candidates['merged'][mergers]
        

    
## Extract merged sequences from fasta


#identify ref contigs for which I need to extract sequences
merge_ref_ctgs = {}
for merged in candidates['merged']:
    ref_ctg = "|".join(merged.split("|")[:-1])
    if not ref_ctg in merge_ref_ctgs:
        merge_ref_ctgs[ref_ctg] = []
    merge_ref_ctgs[ref_ctg].append(merged)
    
#for ctg in sorted(merge_ref_ctgs):
#    print ctg,merge_ref_ctgs[ctg]
    
from Bio import SeqIO

for seq in SeqIO.parse(open(reffasta,'r'),'fasta'):
    if seq.id in merge_ref_ctgs:
#        print seq.id
        for uce in merge_ref_ctgs[seq.id]:
            if candidates['merged'][uce]['ref']['or'] == '+':
                uc_seq = str(seq.seq[candidates['merged'][uce]['ref']['start']-1:candidates['merged'][uce]['ref']['end']-1])
            else:
                uc_seq = str(seq.seq.reverse_complement()[candidates['merged'][uce]['ref']['start']-1:candidates['merged'][uce]['ref']['end']-1])
#            print "\t%s,%s,%s" %(uce, candidates['merged'][uce]['ref']['refstart'], candidates['merged'][uce]['ref']['refend'])
            candidates['merged'][uce]['ref']['seq'] = uc_seq
#            print candidates['merged'][uce]['ref']

            
#Identify query contigs for which I need to extract sequences
merge_que_ctgs = {}
for merged in candidates['merged']:
    ctg = candidates['merged'][merged]['que']['id']
    
    if not ctg in merge_que_ctgs:
        merge_que_ctgs[ctg] = []
    merge_que_ctgs[ctg].append(merged)

    
#for ctg in sorted(merge_que_ctgs):
#    print ctg,merge_que_ctgs[ctg]


for seq in SeqIO.parse(open(quefasta,'r'),'fasta'):
    if seq.id in merge_que_ctgs:
#        print seq.id
        for uce in merge_que_ctgs[seq.id]:
            if candidates['merged'][uce]['que']['or'] == '+':
                uc_seq = str(seq.seq[candidates['merged'][uce]['que']['start']-1:candidates['merged'][uce]['que']['end']-1])
            else:
                uc_seq = str(seq.seq.reverse_complement()[candidates['merged'][uce]['que']['start']-1:candidates['merged'][uce]['que']['end']-1])
#            print "\t%s,%s,%s" %(uce, candidates['merged'][uce]['que']['start'], candidates['merged'][uce]['que']['end'])
            candidates['merged'][uce]['que']['seq'] = uc_seq

#            print "%s\n%s" %(candidates['merged'][uce]['ref']['seq'],candidates['merged'][uce]['que']['seq'])
            
            #test
#            ctg_id = "|".join(uce.split("|")[:-1])
#            for start in to_merge[uce]:
#                print per_ref_ctg[ctg_id][start][0][3]
            

#move invalid matches to bin
for bingroup in ['too-close','overlap-filter','skip']:
#    print bingroup
    for ref_ctg_id in sorted(per_ref_ctg):
    
        for start in per_ref_ctg[ref_ctg_id]:
#            print bingroup,ref_ctg_id,start
#            print per_ref_ctg[ref_ctg_id][start][0]
            temp = per_ref_ctg[ref_ctg_id][start][0]
            
            if bingroup in temp:
                matchid = ref_ctg_id+"|%s" %start
                
                binned[bingroup][matchid] = reformat_match(ref_id=ref_ctg_id, ref_start_pos=start, baselist=temp)

                
#output summary for binned matches
print
for bingroup in binned:
    print "%s\t%s" %(bingroup,len(binned[bingroup]))
    
##summarize candidates
print
for group in candidates.keys():
    print "%s\t%s" %(group,len(candidates[group]))
        

#output length distribution per candidate group
for group in candidates.keys():
    print "\n%s\t%s" %(group,len(candidates[group]))
    length_dist = {}
    for uce in candidates[group]:
        length = candidates[group][uce]['ref']['end'] - candidates[group][uce]['ref']['start']
        if not length in length_dist:
            length_dist[length] = 0
        length_dist[length]+=1
    for l in sorted(length_dist):
        print l,length_dist[l]
#    print candidates[group]



##CHECK distance from contig ends for singlematches too

print "\nWRITING GLOBAL OUTPUTS with prefix: '%s'\n" %global_prefix

write_binned_match_details(global_prefix+'.bin.reason.tsv', master_dict=binned)

write_UCE_coordinates(global_prefix+'.candidate_coordinates.tsv', master_dict=candidates)


name_body = '.candidate_UCEs'
if minlength:
    name_body += '-minlength'+str(minlength)
        
#Extract only UCE's from reference with minimum length of 100
print "\nWRITING: %s%s.fasta" %(ref_prefix,name_body)
extract_UCEs(prefix = ref_prefix+name_body, candidates=candidates, target='ref', minlength=minlength)
#Extract only UCE's from query with minimum length of 100
print "\nWRITING: %s%s.fasta" %(que_prefix,name_body)
extract_UCEs(prefix = que_prefix+name_body, candidates=candidates, target='que', minlength=minlength)

if extension:
    name_body += '-extended'+str(extension)
    #Extract UCE's from reference with minimum length of 100 with 500 bp extension
    print "\nWRITING: %s%s.fasta" %(ref_prefix,name_body)
    extract_UCEs(prefix = ref_prefix+name_body, candidates=candidates, target='ref', minlength=minlength, extension=extension, fasta=reffasta)
    #Extract UCE's from query with minimum length of 100 with 500 bp extension
    print "\nWRITING: %s%s.fasta" %(que_prefix,name_body)
    extract_UCEs(prefix = que_prefix+name_body, candidates=candidates, target='que', minlength=minlength, extension=extension, fasta=quefasta)
    

LvSf80.out.tsv
Total number of MUMMER matches: 1482
Filter 'multiple query hits with identical start position on reference' removed 26 Mummer hits
Leaves us with 1456 hits to process

MERGING


too-close	7
skip	180
overlap-filter	6
merge-dist-diff	1
position	11
irregular-merge	0

unique	153
singlematch	1032
merged	30

unique	153
80 5
81 6
82 5
83 4
84 6
85 5
86 2
87 3
88 2
89 5
90 4
91 2
92 2
93 5
94 4
95 2
97 4
98 4
99 2
100 4
101 1
102 4
103 2
104 2
105 2
106 3
107 1
108 3
109 2
111 1
112 4
113 2
114 3
115 3
116 1
120 2
121 1
124 1
125 1
127 3
128 2
129 1
131 2
132 6
133 2
134 1
136 1
137 2
140 1
141 1
152 1
154 1
157 1
166 1
170 1
175 1
176 1
180 1
195 1
197 1
209 1
221 1
247 2
309 1
392 1

singlematch	1032
80 73
81 54
82 44
83 38
84 32
85 46
86 43
87 37
88 31
89 35
90 24
91 21
92 25
93 21
94 18
95 37
96 25
97 19
98 20
99 12
100 17
101 24
102 15
103 16
104 12
105 9
106 14
107 16
108 13
109 13
110 9
111 8
112 9
113 12
114 6
115 13
116 7
117 7
118 7
119 10
120 4
121 7
122 5
123 9
124 

Combine both output files and sort by reference contig and start position.

In [220]:
%%bash

cat LvSf80.out.candidate_coordinates.tsv LvSf80.out.bin.reason.tsv | sort -k 3,3 -k 4,4n > complete_report.tsv

Output cummulative length distribution.

In [126]:
%%bash
file=Sfran.candidate_UCEs.fasta


for i in {80..300..10}
do 
    cat <(echo $i) $file | \
    grep ">" -v | \
    perl -ne 'chomp; if ($. == 1){ $min = $_; $count=0;}else{if (length($_) >= $min){$count++; print "$min\t$count\n"}}' | \
    tail -n 1
done

80	1214
90	738
100	487
110	314
120	212
130	141
140	109
150	84
160	68
170	56
180	45
190	40
200	27
210	24
220	18
230	15
240	13
250	10
260	9
270	9
280	8
290	7
300	7


In [127]:
%%bash
file=Sfran.candidate_UCEs.fasta

##cummulative length distribution only for merged UCE's
for i in {80..300..10}
do 
    cat <(echo $i) <(cat $file | grep "merged" -A 1 | grep ">" -v) | \
    perl -ne 'chomp; if ($. == 1){ $min = $_; $count=0;}else{if (length($_) >= $min){$count++; print "$min\t$count\n"}}' | \
    tail -n 1
done

80	29
90	29
100	29
110	29
120	29
130	29
140	29
150	29
160	29
170	29
180	26
190	24
200	17
210	16
220	10
230	8
240	6
250	6
260	5
270	5
280	5
290	4
300	4


## The code has been transferred to a python script (01.02.2017) and is running stably now